<h1>FastText Implementation</h1>

In this notebook we apply FastText to our dataset.

In [ ]:
# Needed general imports
import csv, time, os
import numpy as np

# Libraries for FastText
import fasttext
from scipy.sparse import *
from sklearn.model_selection import KFold

<h3>Loading Data</h3>

This function loads data from the processed tweet files, splits the data into words and generates labels. Returns split sentences and labels for the training sets and split sentences for the testing set.

In [ ]:
def load_data_and_labels(positive_data_file, negative_data_file, test_data_file):
    
    # Load data from files
    with open("twitter-datasets/" + positive_data_file, "r", encoding="utf8") as myfile:
        pos_data = [s.strip() for s in list(myfile.readlines())]
        
    with open("twitter-datasets/" + negative_data_file, "r", encoding="utf8") as myfile:
        neg_data = [s.strip() for s in list(myfile.readlines())]
        
    with open("twitter-datasets/" + test_data_file, "r", encoding="utf8") as myfile:
        test_data = [s.strip() for s in list(myfile.readlines())]
    
    # Concatenate data
    train_data = pos_data + neg_data
    
    # Generate labels
    positive_labels = [1 for _ in pos_data]
    negative_labels = [-1 for _ in neg_data]
    
    labels = np.concatenate([positive_labels, negative_labels], 0)
    
    return train_data, labels, test_data

<h3>Classification</h3>

Here we run our classification. We first load the datasets, then save the training set with the labels appended. We feed the resulting file to FastText and get a classifier, which we use to predict labels for the testing set.

In [ ]:
# start computing time 
start = time.time()

# Load data from processed files
train, labels, test = load_data_and_labels('train_pos_proc.txt', 'train_neg_proc.txt', 'test_data.txt')

# Create the correct label in front of every tweets as : '__label__<X>'
with open('../outputs/fastText_labels.txt', 'w', encoding="utf-8") as f:
    for sent, label in zip(train ,labels):
        f.write('__label__' + str(label) + ' ' + sent + '\n')

# define the parameters for the fastText classifier
window, epochs = 10, 10

# Build the fastText classifier 
classifier = fasttext.supervised('outputs/fastText_labels.txt', 'model', label_prefix='__label__', ws=window, epoch=epochs)

# Create the prediction
prediction = classifier.predict_proba(test)

# Compute the computing time
end = time.time()
print(end - start)

<h3>Cross-Validation</h3>

We can now test the accuracy of our classifier by running a 10-fold cross validation on it. With the same parameters as before, we split out data in 10 scrambled subsets, with one of them acting as a testing set in each iteration.

In [ ]:
# Load data from the processed files
train, labels, test = load_data_and_labels('train_pos_proc.txt', 'train_neg_proc.txt', 'test_data.txt')

# define the parameters for the fastText classifier
window, epochs = 10, 10

# create random indices of the rows size
num_row = len(labels)
indices = np.random.permutation(num_row)

# Define the number of folds for the cross-validation
fold = 10;
k_fold = KFold(n_splits=fold)
accuracy = np.zeros((fold))

i = 0

for train_indices, test_indices in k_fold.split(labels):
    
    # Randomize the cross-val indices with the indices created above
    train_indices = indices[train_indices]
    test_indices = indices[test_indices]
    
    # Create the correct label in front of every tweets as : '__label__<X>'
    # For the training set
    with open('outputs/fastText_train_labels.txt', 'w', encoding="utf-8") as f:
        for indice in train_indices:
             f.write('__label__' + str(labels[indice]) + ' ' + train[indice] + '\n')
                
    # For the testing set 
    with open('outputs/fastText_test_labels.txt', 'w', encoding="utf-8") as f:
        for indice in test_indices:
             f.write('__label__'  + str(labels[indice]) + ' ' + train[indice] + '\n')
    
    # Build the fastText classifier 
    classifier = fasttext.supervised('outputs/fastText_train_labels.txt', 'model_cros_val', label_prefix='__label__', ws=window, epoch=epochs)
    
    # Evaluate how the classifier performs on the testing set
    result = classifier.test('outputs/fastText_test_labels.txt')
    
    # Saving the accuracy for every iteration
    accuracy[i] = result.precision
    i += 1

We can now view the accuracy of each of the folds of the cross-validation.

In [ ]:
accuracy

<h3>Submission for FastText</h3>

This code generates a submission file for the FastText implementation.

In [ ]:
with open('outputs/sub_fasttext.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Prediction']
    sub_writer = csv.DictWriter(csvfile, fieldnames)
    index = 0
    sub_writer.writeheader()
    
    for pred in prediction:
        index += 1
        sub_writer.writerow({'Id': str(index), 'Prediction': str(pred[0][0])})